In [1]:
import torch

In [12]:
sample_data = torch.arange(10, dtype=torch.float).view(1,-1)

In [3]:
import torch.nn as nn

In [5]:
kernal_size = 3

filter_conv = nn.Conv1d(1, 1, kernal_size)
gated_conv = nn.Conv1d(1, 1, kernal_size)

In [14]:
filter_conv(sample_data).shape, gated_conv(sample_data).shape

(torch.Size([1, 8]), torch.Size([1, 8]))

In [16]:
import torch.nn.functional as f

In [18]:
f.tanh(filter_conv(sample_data)) * torch.sigmoid(gated_conv(sample_data))

tensor([[ 0.0611, -0.0986, -0.2513, -0.3645, -0.4413, -0.4961, -0.5395, -0.5773]],
       grad_fn=<MulBackward0>)

In [29]:
filter_params = list(filter_conv.parameters())
filter_weights, filter_bias = filter_params[0].data, filter_params[1].data
filter_weights, filter_bias

(tensor([[[-0.1500, -0.4963,  0.2028]]]), tensor([0.2682]))

In [30]:
gated_params = list(gated_conv.parameters())
gated_weights, gated_bias = gated_params[0].data, gated_params[1].data
gated_weights, gated_bias

(tensor([[[ 0.4337, -0.3176,  0.0203]]]), tensor([-0.3524]))

In [36]:
input_size = sample_data.size(-1)

for i in range(input_size):
    print(sample_data[:,i:i+kernal_size])

tensor([[0., 1., 2.]])
tensor([[1., 2., 3.]])
tensor([[2., 3., 4.]])
tensor([[3., 4., 5.]])
tensor([[4., 5., 6.]])
tensor([[5., 6., 7.]])
tensor([[6., 7., 8.]])
tensor([[7., 8., 9.]])
tensor([[8., 9.]])
tensor([[9.]])


In [39]:
for i in range(input_size - kernal_size + 1):
    data = sample_data[:,i:i+kernal_size]
    print(data.squeeze(0)@filter_weights.squeeze(0).T)

tensor([-0.0907])
tensor([-0.5342])
tensor([-0.9777])
tensor([-1.4212])
tensor([-1.8647])
tensor([-2.3082])
tensor([-2.7517])
tensor([-3.1952])


In [41]:
for i in range(input_size - kernal_size + 1):
    data = sample_data[:,i:i+kernal_size]
    print(data.squeeze(0)@filter_weights.squeeze(0).T + filter_bias) 

tensor([0.1775])
tensor([-0.2660])
tensor([-0.7095])
tensor([-1.1530])
tensor([-1.5965])
tensor([-2.0400])
tensor([-2.4835])
tensor([-2.9270])


In [46]:
outputs = torch.tensor([])
for i in range(input_size - kernal_size + 1):
    data = sample_data[:,i:i+kernal_size]
    activation = data.squeeze(0)@filter_weights.squeeze(0).T + filter_bias
    outputs = torch.cat((outputs, activation))

print(outputs)

tensor([ 0.1775, -0.2660, -0.7095, -1.1530, -1.5965, -2.0400, -2.4835, -2.9270])


In [50]:
def conv(inputs, weights, bias):
    input_size = inputs.size(-1)
    
    outputs = torch.tensor([])
    for i in range(input_size - kernal_size + 1):
        data = inputs[:,i:i+kernal_size]
        activation = data.squeeze(0)@weights.squeeze(0).T + bias
        outputs = torch.cat((outputs, activation))
     
    return outputs

In [51]:
conv(sample_data, filter_weights, filter_bias)

tensor([ 0.1775, -0.2660, -0.7095, -1.1530, -1.5965, -2.0400, -2.4835, -2.9270])

In [54]:
torch.tanh(conv(sample_data, filter_weights, filter_bias)) * torch.sigmoid(conv(sample_data, gated_weights, gated_bias))

tensor([ 0.0611, -0.0986, -0.2513, -0.3645, -0.4413, -0.4961, -0.5395, -0.5773])